## 4.1.4 Linear Regression Example with TF-IDF

### TF-IDF Feature Example

In [ ]:
import os

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [ ]:
DEFAULT_PATH = './data_in/' 
DATA_OUT_PATH = './data_out/'

RANDOM_SEED = 42
TEST_SPLIT = 0.2

In [ ]:
train = pd.read_csv(DEFAULT_PATH + "labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

In [ ]:
reviews = list(train['review'])
sentiments = list(train['sentiment'])

In [ ]:
vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3), max_features=5000) #converting data to vectors

X = vectorizer.fit_transform(reviews)
y = np.array(sentiments)

In [ ]:
features = vectorizer.get_feature_names()

In [ ]:
print(X.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

In [ ]:
lgs = LogisticRegression(class_weight='balanced') 
lgs.fit(X_train, y_train) 

In [ ]:
predicted = lgs.predict(X_test)

In [ ]:
fpr, tpr, _ = metrics.roc_curve(y_test, (lgs.predict_proba(X_test)[:, 1]))

In [ ]:
auc = metrics.auc(fpr, tpr)

In [ ]:
print("Accuracy: %f" % lgs.score(X_test, y_test))  
print("Precision: %f" % metrics.precision_score(y_test, predicted))
print("Recall: %f" % metrics.recall_score(y_test, predicted))
print("F1-Score: %f" % metrics.f1_score(y_test, predicted))
print("AUC: %f" % auc)

In [ ]:
test = pd.read_csv(DEFAULT_PATH + "testData.tsv", header=0, delimiter="\t", quoting=3)

In [ ]:
testDataVecs = vectorizer.transform(test['review'])

In [ ]:
test_predicted = lgs.predict(testDataVecs)

In [ ]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

answer_dataset = pd.DataFrame({'id': test['id'], 'sentiment': test_predicted})
answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_tfidf_answer.csv', index=False, quoting=3)